This notebook will be used for the Capstone Project from the Coursera Course <b>Applied Data Science Capstone</b>

In [1]:
import numpy as np
import pandas as pd
import requests

#!conda install -c conda-forge geopy --yes # uncomment this line when not already installed
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line when not already installed
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

In [8]:
# read the statistische Stadtteile from the web
r = requests.get('https://de.wikipedia.org/wiki/Statistische_Gliederung_von_Nürnberg')
c = r.content
dfs = pd.read_html(c)

In [13]:
dfs [1][85:90] # Attention: this table needs to be reformatted because it has in-table headers

,Nr.,Name des statistischen Bezirks/Stadtteils,Fläche (ha)[Anm. 1],Einwohner[Anm. 2],Quelle
,0,Statistischer Stadtteil 0Altstadt und engere Innenstadt,"597,99",58826,[16]
85,86,Buchenbühl,3140,2257,[18]
86,87,Flughafen,3659,-,[18]
87,9,Statistischer Stadtteil 9Östliche Außenstadt,212955,46517,[16]
88,90,St. Jobst,1832,9574,[18]
89,91,Erlenstegen,3436,4009,[18]


# Analyse the neighborhood "Ziegelstein"

In [27]:
#49.4714°, 11.0912°
d= {"Neighborhood": ["Ziegelstein","Marienberg","Schafhof","Mooshof","Buchenbühl","Schoppershof","Schleifweg"],
    "Latitude" : [49.49, 49.47976,49.480095,49.488335,49.504167,49.469444,49.4714 ],
    "Longitude" : [11.103611, 11.090212,11.118321,11.094388,11.11,11.109722,11.0912]
   }
# Latitude of Marienberg manually corrected, was nearly the same as for Schleifweg on Wikipedia

In [25]:
city_data = pd.DataFrame(data= d)
city_data

,Neighborhood,Latitude,Longitude
0,Ziegelstein,49.490000,11.103611
1,Marienberg,49.479760,11.090212
2,Schafhof,49.480095,11.118321
3,Mooshof,49.488335,11.094388
4,Buchenbühl,49.504167,11.110000
5,Schoppershof,49.469444,11.109722
6,Schleifweg,49.471400,11.091200


In [19]:
address = 'Nürnberg, DE'

geolocator = Nominatim(user_agent="de_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Nürnberg are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Nürnberg are 49.453872, 11.077298.


In [26]:
# visualize the location of the neighborhoods within Nürnberg

map_nbg = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(city_data['Latitude'], city_data['Longitude'], city_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nbg)  
    
map_nbg

In [7]:
# configure foursquare
CLIENT_ID = '1ASA1R5SJNI4KKZDE0VT02FRVCZPTLGRT35LWIICH2VGSSYW' # your Foursquare ID
CLIENT_SECRET = 'AQEUCQQX3WF40RDYNTMNFCVRNNV5OY1CC3MCVOSD212UVEKL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1ASA1R5SJNI4KKZDE0VT02FRVCZPTLGRT35LWIICH2VGSSYW
CLIENT_SECRET:AQEUCQQX3WF40RDYNTMNFCVRNNV5OY1CC3MCVOSD212UVEKL


In [8]:
# use function from the Labs
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [62]:
label = city_data.Neighborhood
lat = city_data.Latitude
lng = city_data.Longitude
nbg_venues = getNearbyVenues(label, lat, lng, radius=500)
nbg_venues

Ziegelstein
Marienberg
Schafhof
Mooshof
Buchenbühl
Schoppershof
Schleifweg


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ziegelstein,49.490000,11.103611,"Grüne Au - ""Zum Brez'nwirt""",49.488138,11.105274,German Restaurant
1,Ziegelstein,49.490000,11.103611,Futterstubn,49.489692,11.103054,Burger Joint
2,Ziegelstein,49.490000,11.103611,netlogix GmbH & Co. KG,49.490533,11.106315,IT Services
3,Ziegelstein,49.490000,11.103611,Alpha Hotel,49.488389,11.105442,Hotel
4,Ziegelstein,49.490000,11.103611,NORMA,49.487058,11.103422,Supermarket
5,Ziegelstein,49.490000,11.103611,Sport Park Nord,49.488872,11.097466,Gym / Fitness Center
6,Marienberg,49.479760,11.090212,Volkspark Marienberg,49.479428,11.093525,Park
7,Marienberg,49.479760,11.090212,Gaststätte Marienberg,49.477480,11.091650,Italian Restaurant
8,Schafhof,49.480095,11.118321,Café Kraft,49.479874,11.122704,Climbing Gym
9,Schafhof,49.480095,11.118321,Ofenwerk,49.477037,11.118181,Event Space


In [65]:
nbg_venues["Venue"].value_counts()

Sport Park Nord                         2
Lotto Bayern                            2
Grüne Au - "Zum Brez'nwirt"             1
Tennisclub Noris                        1
netlogix GmbH & Co. KG                  1
Der Beck                                1
Bahnhof Nürnberg Nordost                1
Ofenwerk                                1
Landbierparadies                        1
Slotrace & More Spielwarenhandel GbR    1
Kalchreuther Bäcker                     1
PUMA Outlet                             1
Alpha Hotel                             1
Club am Marienberg e.V.                 1
Futterstubn                             1
Bayerischer Staatsforst Buchenbühl      1
Tucherland                              1
NORMA                                   1
Volkspark Marienberg                    1
H Schopenhauerstraße                    1
Gasthof Saalbau Buchenbühl              1
BACK.BUDE                               1
Osteria da Silvano                      1
Gaststätte Marienberg             

In [64]:
nbg_venues["Venue Longitude"].nunique()

29

In [57]:
nbg_venues.loc[1,"Venue Longitude"]

11.105127053845434

Issues:
if radius is too less then only few venues are found in each neighborhood
if radius is too big, there is too much overlapping in the venues found
